## Imports

In [2]:
import os
import sys
from pathlib import Path

sys.path.extend(['/Users/fabianloewe/Projects/master-thesis/tools/aletheia'])

import pandas as pd
import aletheialib.attacks as attacks
import aletheialib.options.tools as tools

## Prepare data

In [9]:
EMBEDDING_METHODS = [
    'MobiStego', 'Passlok', 'Pictograph', 'PixelKnot', 'PocketStego', 'SteganographyM'
]

BASE_PATH = Path('../master-thesis/datasets/StegoAppDB_stegos_20240309-030352')

df = pd.read_csv(BASE_PATH / 'StegoAppDB_stegos_20240309-030352_stego_directory.csv')
dfs = {m: df[df['embedding_method'] == m] for m in EMBEDDING_METHODS}
dfs_image_pairs = {
    m: list(
        (BASE_PATH / 'covers' / ci, BASE_PATH / 'stegos' / si)
        for ci, si in zip(dfs[m]['cover_image_filename'], dfs[m]['image_filename'])
    )
    for m in EMBEDDING_METHODS
}

In [10]:
pd.DataFrame(((m, len(pairs)) for m, pairs in dfs_image_pairs.items()), columns=['Method', 'Count'])

,Method,Count
0,MobiStego,3060
1,Passlok,1530
2,Pictograph,4800
3,PixelKnot,3060
4,PocketStego,3060
5,SteganographyM,3060


## Create metadata diffs

In [11]:
def _merge_dicts(first, second):
    new_dict = {}
    for key in first.keys() | second.keys():
        if key in first and key in second and first[key][1] == second[key][1]:
            new_dict[key] = first[key]
        elif key in first and key not in second:
            new_dict[key] = first[key]
        elif key in second and key not in first:
            new_dict[key] = second[key]
    return new_dict


for method, image_pairs in dfs_image_pairs.items():
    print(method)
    try:
        intermediate = attacks.metadata_diff(image_pairs[0][0], image_pairs[0][1])
    except Exception as e:
        intermediate = {}

    for cover, stego in image_pairs[1:]:
        try:
            diff = attacks.metadata_diff(cover, stego)
            print(stego, '->', diff)
            intermediate = _merge_dicts(intermediate, diff)
        except Exception as e:
            print(e)
    intermediate = {k: v[1] for k, v in intermediate.items() if k != 'Size'}
    print(intermediate)

MobiStego
/Users/fabianloewe/Projects/master-thesis/datasets/StegoAppDB_stegos_20240309-030352/stegos/260509.PNG -> {'Mode': ('RGB', 'RGBA'), 'Size': (289351, 326445)}
/Users/fabianloewe/Projects/master-thesis/datasets/StegoAppDB_stegos_20240309-030352/stegos/260510.PNG -> {'Mode': ('RGB', 'RGBA'), 'Size': (289351, 339390)}
/Users/fabianloewe/Projects/master-thesis/datasets/StegoAppDB_stegos_20240309-030352/stegos/260511.PNG -> {'Mode': ('RGB', 'RGBA'), 'Size': (289351, 352487)}
/Users/fabianloewe/Projects/master-thesis/datasets/StegoAppDB_stegos_20240309-030352/stegos/260512.PNG -> {'Mode': ('RGB', 'RGBA'), 'Size': (289351, 366049)}
/Users/fabianloewe/Projects/master-thesis/datasets/StegoAppDB_stegos_20240309-030352/stegos/260822.PNG -> {'Mode': ('RGB', 'RGBA'), 'Size': (247694, 256190)}
/Users/fabianloewe/Projects/master-thesis/datasets/StegoAppDB_stegos_20240309-030352/stegos/260823.PNG -> {'Mode': ('RGB', 'RGBA'), 'Size': (247694, 269042)}
/Users/fabianloewe/Projects/master-thesis/